1. Load CSVs containing Iris training/test data into a TensorFlow Dataset
2. Construct a neural network classifier
3. Fit the model using the training data
4. Evaluate the accuracy of the model
5. Classify new samples

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from urllib.request import urlopen

import tensorflow as tf
import numpy as np

IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"


Then, if the training and test sets aren't already stored locally, download them.

In [13]:
if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read().decode("utf-8") 
    with open(IRIS_TRAINING,'w') as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read().decode("utf-8") 
    with open(IRIS_TEST,'w') as f:
        f.write(raw)

In [16]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
        filename=IRIS_TRAINING,
        target_dtype=np.int,
        features_dtype=np.float32)

In [23]:
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [27]:
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)] # features feed

In [30]:
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                            hidden_units = [10,20,10],
                                            n_classes = 3,
                                            model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f02761b2630>, '_task_type': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_keep_checkpoint_max': 5, '_model_dir': '/tmp/iris_model', '_tf_random_seed': None, '_task_id': 0, '_is_chief': True, '_num_worker_replicas': 0, '_save_summary_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_evaluation_master': ''}


In [35]:
# Define the training inputs
def get_train_inputs():
    
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    
    return x, y

In [44]:
print(classifier.fit(input_fn=get_train_inputs, steps=2000))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-4000
INFO:tensorflow:Saving checkpoints for 4001 into /tmp/iris_model/model.ckpt.
INFO:tensorflow:step = 4001, loss = 0.0340775
INFO:tensorflow:global_step/sec: 1640.44
INFO:tensorflow:step = 4101, loss = 0.0337386 (0.061 sec)
INFO:tensorflow:global_step/sec: 1698.97
INFO:tensorflow:step = 4201, loss = 0.0334072 (0.059 sec)
INFO:tensorflow:global_step/sec: 1746.6
INFO:tensorflow:step = 4301, loss = 0.033083 (0.057 sec)
INFO:tensorflow:global_step/sec: 1828.36
INFO:tensorflow:step = 4401, loss = 0.032766 (0.055 sec)
INFO:tensorflow:global_step/sec: 1830

In [37]:
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)

    return x, y

In [38]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                     steps=1)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-07-20-08:21:04
INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-07-20-08:21:04
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, global_step = 2000, loss = 0.0617994

Test Accuracy: 0.966667



In [42]:
def new_samples():
    return np.array([[6.4, 3.2, 4.5, 1.5],
                     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

predictions = list(classifier.predict_classes(input_fn=new_samples))

print("New Samples, Class Predictions:    {}\n".format(predictions))

INFO:tensorflow:Restoring parameters from /tmp/iris_model/model.ckpt-2000
New Samples, Class Predictions:    [1, 2]

